In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from pyannote.core import SlidingWindowFeature

In [4]:

from audio.redis import *
from audio.audio import *
from audio.process import *

from pathlib import Path
import json

import uuid

import pandas as pd

import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

import json

In [5]:
with open('../header', 'rb') as f:
    header = f.read()

In [6]:

redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [9]:
[await redis_stream_client.delete(c) for c in conns['queue'].tolist()]

[]

In [13]:
conns = await get_connection_queues(name = 'initialFeed_audio', pattern = "*",redis_client= redis_stream_client)
conns

,queue,length
0,initialFeed_audio:5a3f82ca-eb7f-406a-b7cb-738b...,2


In [14]:
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [15]:
conn_id = conns['queue'].iloc[0].split(':')[1]

In [16]:
conn = ApiConnection(conn_id,redis_stream_client)

In [18]:
duration = 10
for _ in range(duration):
    _, item = await redis_stream_client.brpop(conn.queue)
    chunk = bytes.fromhex(json.loads(item)['chunk'])
    # Open the file in append mode
    with open('audio.webm', 'ab') as file:
        # Write data to the file
        file.write(chunk)

In [166]:
import subprocess
from pydub import AudioSegment

In [167]:
tail_duration = 10

In [168]:
webm_file_path = 'audio.webm'

In [169]:
import io

In [156]:
start_time = 0

In [157]:
ffmpeg_command = ['ffmpeg', '-ss', str(start_time), '-t', str(tail_duration),
                  '-i', webm_file_path, '-f', 'mp3', '-acodec', 'libmp3lame', '-']

# Run the ffmpeg command and capture the output
result = subprocess.run(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Use BytesIO to create a file-like object from the output
audio_data = io.BytesIO(result.stdout)

# Load this audio data into pydub for further manipulation if needed
audio_segment = AudioSegment.from_file(audio_data, format="mp3")


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --

In [158]:
audio_data

In [61]:
audio_segment

In [100]:
chunks = await conn.pop_multiple(12,12)

In [83]:
len(b''.join([c.chunk for c in chunks]))

196526

In [102]:
b''.join([c.chunk for c in chunks])

b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87\xb6\xe23\xfa3qS\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x02\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x02bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3D\xbc\x81\x00\x00\x80\xff\x83\xff<\xfc3\x7f\xd1\x037M0\x8f1\xf6\x9fR}\xa2\x1enl\x8d\x0co\xba\xfcCm\xffZ\x82\xae\xf6$\x16\x9e\xc5]\xbcHJ\x94UL\x19\xff\xcfM\xb9\xc7^\xc7r\xef\x1b\xbd\xb2l\x1d\x86!\xe9L\x8b\xc9^\x18\xe3\xdd\xb35\x0e\xc9\x88\xdeO\x93\xbb)^$"\xc5\xa3\xad\xac\x83\x0fY\\\tZ\xd6\x971p\x0b}{\x02\xa6;j,\xa7\x8f8]\r\x12CR\x872\x13\x8f\x90z\xc9C}$\xaaD\xce\xcc\xac\x8f\x8c[\xb9C\x847)A\xbeW\x0e\x0b\xd96.(W;_HDoh\x96\xb9\xe6\xe3\xff\x1aV\x1fE\'\xc7>:\x08\x1a\xcd\xdb\xe2m\xc8\x00?\xd6\x00\xa2\t\x8bl(g\xa5+\xf49\xe0vBS8y\x87\x86\xcd

In [88]:
len(b''.join([c.chunk for c in chunks]))

198333

In [91]:
len(b''.join([c.chunk for c in chunks]))

198128

In [47]:
audio_slicer1 = AudioSlicer(data=(header+b''.join([c.chunk for c in chunks])),format='webm')

In [48]:
audio_slicer.slice()

None


In [51]:
audio_slicer2 = AudioSlicer(data=(header+b''.join([c.chunk for c in chunks])),format='webm')

In [61]:
audio_slicer2.audio

In [56]:
audioslicer = (audio_slicer1.audio+audio_slicer2.audio)

In [59]:
audioslicer

In [33]:
chunks = await conn.pop_multiple(12,12)

In [35]:
await audio_slicer.append(header+b''.join([c.chunk for c in chunks]))

In [36]:
audio_slicer.slice()

None


In [30]:
chunks = await conn.pop_multiple(20,20)

In [35]:
audio_slicer.append_raw_audio(b''.join([c.chunk for c in chunks]))

In [36]:
audio_slicer.slice()

None


In [503]:
frame_rate=48000
channels=1
sample_width=2
AudioSegment.from_raw(file=io.BytesIO(b''.join([c.chunk for c in chunks])),frame_rate=frame_rate,channels=channels,sample_width=sample_width)

In [492]:
audio_slicer.slice()

None


In [199]:
await audio_slicer.append(header+b''.join([c.chunk for c in chunks]))

In [200]:
audio_slicer.slice()

None


In [ ]:
try:
    while True:
        print(start,end)
        audio_data = await audio_slicer.export_data(start,end)
        audio_name = str(uuid.uuid4())
        audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
        await audio.save()
        print('waiting diarize')
        diarization_result = await diarize(audio_data,client_id,audio_name,start)
        print('waiting transcribe')
        await transcribe(diarization_result,connection_id,audio_name)
        start_ = await get_next_chunk_start(diarization_result,length)
        start = start_ if start_ else start+length
        if end>audio_slicer.audio.duration_seconds:break
        end+=length
except KeyboardInterrupt:
    pass